In [3]:
# 1. 필요한 라이브러리 및 크롬 드라이버 설치 (약 1분 소요)
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:3 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,208 B]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:13 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 3,033 B in 1s (2,207 B/s)
Reading package lists... Done
W: Skipping a

In [25]:
# 1. 필수 라이브러리 설치 (최초 1회 실행)
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- 1. 브라우저 옵션 설정 (차단 방지) ---
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # 화면 없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# 봇 탐지 우회를 위한 User-Agent 설정
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36')

driver = webdriver.Chrome(options=options)

# --- 2. 타겟 설정 ---
CLUB_ID = "10094499"  # 카페 ID
MENU_ID = "618"       # 요청하신 메뉴 ID (110호 기모바지 글이 있던 게시판)
START_PAGE = 21
END_PAGE = 60          # 원하는 페이지 수만큼 늘리세요

results = []

print(f"🚀 메뉴 ID {MENU_ID}번 크롤링을 시작합니다...")

try:
    for page in range(START_PAGE, END_PAGE + 1):
        # [핵심] 모바일 URL 대신 크롤링하기 좋은 PC 버전 URL로 재구성
        url = f"https://cafe.naver.com/ArticleList.nhn?search.clubid={CLUB_ID}&search.menuid={MENU_ID}&search.boardtype=L&search.page={page}"

        driver.get(url)

        try:
            # 게시판 리스트가 뜰 때까지 대기
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.article-board"))
            )

            # a.article 클래스를 가진 모든 요소 찾기 (질문하신 부분)
            # 네이버 카페 PC 버전 리스트에서 제목은 'a.article' 클래스를 가집니다.
            articles = driver.find_elements(By.CSS_SELECTOR, "a.article")

            count = 0
            for article in articles:
                # 텍스트 추출 (제목)
                title_text = article.text.strip()

                # 링크 주소 추출 (href)
                link_url = article.get_attribute("href")

                if title_text: # 빈 텍스트가 아닐 경우만 저장
                    results.append({
                        "페이지": page,
                        "제목": title_text,
                        "링크": link_url
                    })
                    count += 1

            print(f"✅ {page}페이지: {count}개 게시글 수집 완료")
            time.sleep(1) # 차단 방지용 대기

        except Exception as e:
            print(f"❌ {page}페이지 수집 실패: {e}")
            continue

except Exception as e:
    print(f"시스템 오류: {e}")

finally:
    driver.quit()

# --- 3. 결과 저장 및 확인 ---
if results:
    df = pd.DataFrame(results)
    # 엑셀 파일로 저장
    df.to_csv(f"naver_cafe_menu_{MENU_ID}_3.csv", index=False, encoding="utf-8-sig")

    print("\n[수집 결과 미리보기]")
    print(df.head())
else:
    print("\n수집된 데이터가 없습니다.")

🚀 메뉴 ID 618번 크롤링을 시작합니다...
✅ 21페이지: 15개 게시글 수집 완료
✅ 22페이지: 15개 게시글 수집 완료
✅ 23페이지: 15개 게시글 수집 완료
✅ 24페이지: 15개 게시글 수집 완료
✅ 25페이지: 15개 게시글 수집 완료
✅ 26페이지: 15개 게시글 수집 완료
✅ 27페이지: 15개 게시글 수집 완료
✅ 28페이지: 15개 게시글 수집 완료
✅ 29페이지: 15개 게시글 수집 완료
✅ 30페이지: 15개 게시글 수집 완료
✅ 31페이지: 15개 게시글 수집 완료
✅ 32페이지: 15개 게시글 수집 완료
✅ 33페이지: 15개 게시글 수집 완료
✅ 34페이지: 15개 게시글 수집 완료
✅ 35페이지: 15개 게시글 수집 완료
✅ 36페이지: 15개 게시글 수집 완료
✅ 37페이지: 15개 게시글 수집 완료
✅ 38페이지: 15개 게시글 수집 완료
✅ 39페이지: 15개 게시글 수집 완료
✅ 40페이지: 15개 게시글 수집 완료
✅ 41페이지: 15개 게시글 수집 완료
✅ 42페이지: 15개 게시글 수집 완료
✅ 43페이지: 15개 게시글 수집 완료
✅ 44페이지: 15개 게시글 수집 완료
✅ 45페이지: 15개 게시글 수집 완료
✅ 46페이지: 15개 게시글 수집 완료
✅ 47페이지: 15개 게시글 수집 완료
✅ 48페이지: 15개 게시글 수집 완료
✅ 49페이지: 15개 게시글 수집 완료
✅ 50페이지: 15개 게시글 수집 완료
✅ 51페이지: 15개 게시글 수집 완료
✅ 52페이지: 15개 게시글 수집 완료
✅ 53페이지: 15개 게시글 수집 완료
✅ 54페이지: 15개 게시글 수집 완료
✅ 55페이지: 15개 게시글 수집 완료
✅ 56페이지: 15개 게시글 수집 완료
✅ 57페이지: 15개 게시글 수집 완료
✅ 58페이지: 15개 게시글 수집 완료
✅ 59페이지: 15개 게시글 수집 완료
✅ 60페이지: 15개 게시글 수집 완료

[수집 결과 미리보기]
   페이지                                 

In [22]:
!pip install bertopic pandas
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 10.3 MB/s eta 0:00:00


In [24]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# -----------------------------------------------------------
# 1. 데이터 불러오기
# -----------------------------------------------------------
# 이전에 크롤링해서 저장한 파일명을 정확히 입력해라
file_name = "naver_cafe_menu_618_2.csv"

# CSV 읽기 (에러 방지를 위해 문자열로 변환)
df = pd.read_csv(file_name)
docs = df.iloc[:, 0].astype(str).tolist() # 첫 번째 컬럼(제목)을 리스트로 변환

print(f"데이터 로드 완료: 총 {len(docs)}개 문서")
print(f"예시 데이터: {docs[:3]}")

# -----------------------------------------------------------
# 2. 모델 설정 및 학습 (핵심)
# -----------------------------------------------------------
print("🚀 BERT 모델 로딩 및 토픽 모델링 시작... (시간이 좀 걸린다)")

# 1) 임베딩 모델 선택: 한국어를 지원하는 다국어 모델 사용
# "paraphrase-multilingual-MiniLM-L12-v2"는 빠르고 성능이 준수함
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# 2) BERTopic 객체 생성
# min_topic_size: 토픽 하나당 최소 글 개수 (데이터가 적으면 5~10으로 줄여라)
topic_model = BERTopic(embedding_model=sentence_model,
                       min_topic_size=5,
                       calculate_probabilities=True,
                       verbose=True)

# 3) 학습 및 토픽 추출
topics, probs = topic_model.fit_transform(docs)

# -----------------------------------------------------------
# 3. 결과 확인
# -----------------------------------------------------------

# 생성된 토픽 정보 출력
freq = topic_model.get_topic_info()
print("\n[토픽 모델링 결과 요약]")
print(freq.head(10)) # 상위 10개 토픽 출력

# 특정 토픽(예: 0번 토픽)의 주요 키워드 확인
print("\n[가장 비중이 큰 토픽 0의 키워드]")
print(topic_model.get_topic(0))

# -----------------------------------------------------------
# 4. 시각화 및 저장
# -----------------------------------------------------------

# 1) 토픽 간 거리 지도 (Intertopic Distance Map)
# 토픽들이 서로 얼마나 비슷한지 보여주는 시각화
fig1 = topic_model.visualize_topics()
fig1.write_html("bertopic_visualization.html")

# 2) 토픽별 막대 그래프 (Barchart)
fig2 = topic_model.visualize_barchart(top_n_topics=10)
fig2.write_html("bertopic_barchart.html")

print("\n🎉 분석 완료! 'bertopic_visualization.html' 파일을 다운로드해서 열어봐.")

데이터 로드 완료: 총 344개 문서
예시 데이터: ['1', '1', '1']
🚀 BERT 모델 로딩 및 토픽 모델링 시작... (시간이 좀 걸린다)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-11-22 08:45:32,130 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2025-11-22 08:45:33,895 - BERTopic - Embedding - Completed ✓
2025-11-22 08:45:33,897 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-22 08:45:45,639 - BERTopic - Dimensionality - Completed ✓
2025-11-22 08:45:45,640 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-22 08:45:45,698 - BERTopic - Cluster - Completed ✓
2025-11-22 08:45:45,713 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-22 08:45:45,751 - BERTopic - Representation - Completed ✓



[토픽 모델링 결과 요약]
   Topic  Count     Name          Representation Representative_Docs
0      0     59    0____    [, , , , , , , , , ]           [1, 1, 1]
1      1     15    1____    [, , , , , , , , , ]           [2, 2, 2]
2      2     15    2____    [, , , , , , , , , ]           [3, 3, 3]
3      3     15    3____    [, , , , , , , , , ]           [4, 4, 4]
4      4     15    4____    [, , , , , , , , , ]           [5, 5, 5]
5      5     15    5____    [, , , , , , , , , ]           [6, 6, 6]
6      6     15    6____    [, , , , , , , , , ]           [7, 7, 7]
7      7     15    7____    [, , , , , , , , , ]           [8, 8, 8]
8      8     15    8____    [, , , , , , , , , ]           [9, 9, 9]
9      9     15  9_10___  [10, , , , , , , , , ]        [10, 10, 10]

[가장 비중이 큰 토픽 0의 키워드]
[('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]

🎉 분석 완료! 'bertopic_visualization.html' 파일을 다운로드해서 열어봐.
